In [54]:
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.multiclass import OneVsRestClassifier
import pandas as pd
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

    

In [2]:
train =  pd.read_csv("train.csv", index_col=None)

In [3]:
train.head()

,business_id,categories,city,full_address,latitude,longitude,name,neighborhoods,open,review_count,stars_x,state,type_x,date,review_id,stars_y,text,type_y,user_id,votes
0,rncjoVoEFUJGCUoC1JgnUA,"['Accountants', 'Professional Services', 'Tax ...",Peoria,"8466 W Peoria Ave\nSte 6\nPeoria, AZ 85345",33.581867,-112.241596,Peoria Income Tax Service,[],True,3,5.0,AZ,business,2012-02-12,EnAdKZ_u_wj9ifTRwkfVwg,5,I highly reccomend this place. They helped my ...,review,HK35ai8frY75iMYBVdD_Pg,"{'funny': 0, 'useful': 1, 'cool': 0}"
1,rncjoVoEFUJGCUoC1JgnUA,"['Accountants', 'Professional Services', 'Tax ...",Peoria,"8466 W Peoria Ave\nSte 6\nPeoria, AZ 85345",33.581867,-112.241596,Peoria Income Tax Service,[],True,3,5.0,AZ,business,2012-01-03,QkIrHs6Qb9aEPDXi-ulIsw,5,I received great tax preparation service at a ...,review,EtQsLrSaXqWZN8i8Y_zB-A,"{'funny': 0, 'useful': 1, 'cool': 0}"
2,rncjoVoEFUJGCUoC1JgnUA,"['Accountants', 'Professional Services', 'Tax ...",Peoria,"8466 W Peoria Ave\nSte 6\nPeoria, AZ 85345",33.581867,-112.241596,Peoria Income Tax Service,[],True,3,5.0,AZ,business,2012-02-12,fDg_tMd2rGU2dwxzL6V7kQ,5,I'll never use another cheapy online tax progr...,review,PahwPVfd6BkDyO8KCFEoWw,"{'funny': 0, 'useful': 1, 'cool': 0}"
3,0FNFSzCFP_rGUoJx8W7tJg,"['Sporting Goods', 'Bikes', 'Shopping']",Phoenix,"2149 W Wood Dr\nPhoenix, AZ 85029",33.604054,-112.105933,Bike Doctor,[],True,5,5.0,AZ,business,2011-06-08,yEQacRqY2MJRHTkrUJYLOw,5,Love love the quick repairs and great service....,review,_MucjBvznq2y3yDEt9rOHA,"{'funny': 0, 'useful': 0, 'cool': 0}"
4,0FNFSzCFP_rGUoJx8W7tJg,"['Sporting Goods', 'Bikes', 'Shopping']",Phoenix,"2149 W Wood Dr\nPhoenix, AZ 85029",33.604054,-112.105933,Bike Doctor,[],True,5,5.0,AZ,business,2012-09-21,muIhE1HQZscwe06ISMu81A,5,I couldn't have asked for a better experience!...,review,szbTVEtATfn8f9GxbGGO5w,"{'funny': 0, 'useful': 0, 'cool': 0}"


In [6]:
len(train)

229907

In [35]:
train = train[['text','categories']]

#subset data to test model
train = train.sample(frac=0.1)
len(train)

22991

In [12]:
X_train = np.array(train['text'].values.astype('U'))
y_train =  np.array(train['categories'])

In [13]:
classifier = Pipeline([
    #Convert a collection of text documents to a matrix of token counts
    ('vectorizer', CountVectorizer(ngram_range=(1,1))),
    #TFIDF
    ('tfidf', TfidfTransformer()),
    ('clf', OneVsRestClassifier(LinearSVC()))])


In [14]:
classifier.fit(X_train,y_train )



Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
       ...lti_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
          n_jobs=1))])

In [15]:
#Save model
import pickle
pickle.dump(classifier, open('OneVsRestClassifier_10%subset', 'wb'))

In [ ]:
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))


In [44]:
test =  pd.read_csv("test.csv", index_col=None)
print(len(test))
test = test[['text','categories']]
#subset data to test model
test = test.sample(frac=0.1)
print(len(test))

X_test = np.array(test['text'].values.astype('U'))
y_test =  np.array(test['categories'])


10702
1070


In [46]:
predicted = classifier.predict(X_test)



In [50]:
predicted

array(["['Pizza', 'Restaurants']", "['Burgers', 'Restaurants']",
       "['Barbers', 'Beauty & Spas']", ..., "['Local Flavor']",
       "['Mexican', 'Restaurants']", "['Mexican', 'Restaurants']"],
      dtype='<U181')

In [52]:
y_test

array(["['Pizza', 'Restaurants']",
       "['Burgers', 'Fast Food', 'American (Traditional)', 'Restaurants']",
       "['Barbers', 'Beauty & Spas']", ..., "['Restaurants', 'Italian']",
       "['Mexican', 'Restaurants']", "['Mexican', 'Restaurants']"], dtype=object)

In [51]:
X_test

array([ "I found this place by using Yelp and being a Vegan that hates Daiya cheese this was the place to be. They use follow you heart even though it is more expensive for them to get it. \n\nMe and my non Vegan boyfriend absolutely loved this dining experience. He could not tell that the cheese was a soy cheese, the pizza crust was awesome and the live acoustic music was wonderful.\n\nThe manager/ owner came by our table and many other tables to check and see how we liked the food and asked even if we had suggestions but we didn't ... everything was perfect. I will be back very soon.",
       'Always great!  Great service and yummy vegetarian options. One of the best veggie burgers around. \nJosh is an awesome server!',
       'OK, sorry for the delay in updating since visiting Good Fellas several months back.\n\nFirst off, if you do not reach someone live on the phone when you call, keep calling!\n\nDo not expect to receive a call back if you leave a message, \n\nOnce you manage to 

In [64]:
#Micro has implications -- better to predict and hit occasionally than to never predict at all
f1 = f1_score(y_test,predicted, average ='micro')
recall =  recall_score(y_test, predicted, average= 'micro')
precission =precision_score(y_test, predicted, average='micro')

In [63]:
print("f1: {}".format(f1))
print("precission: {}".format(precission))
print("recall: {}".format(recall))

f1: 0.3177570093457944
precission: 0.3177570093457944
recall: 0.3177570093457944


In [66]:
recall-f1

0.0